In [1]:
import os
import pandas
import psycopg2

region = os.getenv('AWS_REGION')


df = pandas.read_parquet('../data/bger-2024-3-text.parquet')


# Notes for porting to AWS Serverless

Recommendation: Start with AWS Lambda
For your use case (encoding a single string with distiluse-base-multilingual-cased-v2), AWS Lambda is the better choice because:
It’s more cost-effective for low-to-moderate traffic.

The model size (135M parameters) and task (encoding a single string) fit within Lambda’s memory and timeout constraints.

Using a container image and ONNX optimization simplifies deployment and reduces latency.

Steps to Get Started:
Convert the model to ONNX to reduce size and inference time.

Build a Docker image with the model and sentence-transformers.

Deploy to Lambda with 1024-2048 MB memory and a 60-second timeout.

Test with a sample invocation and monitor cold start performance.

If latency is an issue, consider SageMaker Serverless or Provisioned Concurrency.

If you encounter issues (e.g., package size or library conflicts), refer to the Stack Overflow discussion on Lambda deployment errors and ensure a CPU-only PyTorch build.



In [2]:

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  },
  {
    "title": "Agissements potentiellement illégaux de la société TVT",
    "content": "Un avis, intitulé « Avis de travaux », de la société TVT a été affiché à l'entrée d'un immeuble situé dans la commune d'Ecublens, canton de Vaud. Un résident de cet immeuble s'inquiète de la véracité de cet avis, relevant que ni les locataires, ni la gérance, ni les propriétaires n'ont commandé de travaux. L'avis prétend également que « L'installation est obligatoire et gratuite », une affirmation pour laquelle le résident ne perçoit aucune base légale. Au-delà d'une simple pratique commerciale agressive, le résident craint que la société TVT ne se prévale de l'autorité communale pour s'introduire par la ruse dans les propriétés privées et se demande si de telles actions sont punies pénalement en vertu du droit suisse."
  }
]


In [7]:
from pathlib import Path
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer
onnx_path = Path("../data/onnx_model").resolve()
tokenizer = AutoTokenizer.from_pretrained(str(onnx_path))

# Load ONNX model directly
session = ort.InferenceSession(str(onnx_path / "model.onnx"))

# Prepare inputs
case_number = 1
sentence = test_cases[case_number]["content"]
inputs = tokenizer(sentence, return_tensors="np")

# Rename input keys if needed (depends on your export)
onnx_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}

# Run inference
outputs = session.run(None, onnx_inputs)[0]
sentence_embedding = outputs[0]

# Use embedding for search
conn = psycopg2.connect(
    host=os.getenv("AWS_AURORA_READER_ENDPOINT"),
    port=5432,
    dbname="fedcourtdecisions",
    user=os.getenv("AWS_AURORA_VECTORS_USERNAME"),
    password=os.getenv("AWS_AURORA_VECTORS_PASSWORD")
)
# Convert embedding in PostgreSQL vector format
pg_vector = "[" + ",".join(f"{x:.6f}" for x in sentence_embedding.tolist()) + "]"

# pgvector request
with conn.cursor() as cur:
    cur.execute(f"""
        SET ivfflat.probes = 20;
        SELECT doc_id
        FROM embeddings
        ORDER BY embedding <-> %s::vector
        LIMIT 10;
    """, (pg_vector, ))

    hit_ids = {row[0] for row in cur.fetchall()}
    print("Matched doc_ids:", hit_ids)

conn.close()

relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

Matched doc_ids: {'1B_226/2012', '1B_103/2018', '9C_83/2023', '9C_259/2016', '2D_34/2020', '2C_144/2007', '9C_692/2020', '8C_166/2020', '2C_893/2022', '1B_485/2022'}


[{'doc': 'Tribunale federale Tribunal federal {T 0/2} 2C_144/2007 /zga Urteil vom 29. August 2007 II. öffentlich-rechtliche Abteilung Besetzung Bundesrichter Merkli, Präsident, Bundesrichter Hungerbühler, Wurzburger, Gerichtsschreiber Moser. Parteien A.Y. und B.Y.________, Beschwerdeführer, vertreten durch Advokat Stephan Bläsi, gegen Sicherheitsdepartement des Kantons Basel-Stadt, Bereich Recht, Appellationsgericht des Kantons Basel-Stadt als Verwaltungsgericht. Gegenstand Standplatzzuteilung für die Herbstmesse 2006 (Riesenrad), Beschwerde in öffentlich-rechtlichen Angelegenheiten gegen das Urteil des Appellationsgerichts des Kantons Basel-Stadt als Verwaltungsgericht vom 10. Januar 2007. Sachverhalt: A. A.Y. und B.Y.________, welche ein rund 60 m grosses Riesenrad betreiben, erhielten in den vergangenen Jahren hierfür wiederholt einen Standplatz an der Basler Herbstmesse auf dem Münsterplatz. Ihr am 30. November 2005 gestelltes Gesuch für einen Standplatz an der Basler Herbstmesse 2

In [8]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Décision {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des décisions fédérales fournies, voici un avis juridique concernant la situation d’un ingénieur software allemand demandeur d’emploi en Suisse, qui allègue une discrimination en raison de sa nationalité lors du recrutement par une entreprise semi-publique suisse.

1. **Cadre juridique applicable**

- L’Accord sur la libre circulation des personnes (ALCP) entre la Suisse et l’Union européenne interdit toute discrimination fondée sur la nationalité dans le domaine de l’emploi (cf. Décision 9C_259/2016, consid. 5.4.2.2).
- L’article 8 al. 2 de la Constitution fédérale suisse garantit également l’interdiction de la discrimination, notamment fondée sur la nationalité (cf. Décision 2D_34/2020, consid. 2.1).
- Le Tribunal fédéral applique ces principes en exigeant que les situations comparables soient traitées de manière égale, et que toute différence de traitement fondée sur la nationalité soit justifiée par un motif objectif légitime, nécessaire et proportionné (cf. Décision 2D

In [3]:
df

,docref,url,date,year,proc_type,merged_cases,division,division_type,n_judges,language,...,resp_represented,outcome,outcome_binary,cited_bger,n_cited_bger,cited_bge,n_cited_bge,leading_case,text,doi_version
0,1B_8/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Criminal,False,1. Public,Public,3.0,de,...,False,rejected,False,1B_8/2007,1,129 I 361;127 II 32;129 I 361;123 I 268,3,None,Tribunale federale Tribunal federal {T 0/2} 1B...,10.5281/zenodo.14867950
1,1B_19/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Criminal,False,1. Public,Public,3.0,fr,...,False,rejected,False,1B_19/2007;1S.1/2006,2,123 I 268;124 I 336;116 Ia 143;123 I 31;116 Ia...,8,None,Tribunale federale Tribunal federal {T 0/2} 1B...,10.5281/zenodo.14867950
2,1C_16/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Public,True,1. Public,Public,1.0,de,...,False,inadmissible,False,1C_16/2007;1C_18/2007,2,,0,None,Tribunale federale Tribunal federal {T 0/2} 1C...,10.5281/zenodo.14867950
3,5A_39/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-27,2007.0,Civil,False,2. Civil,Civil,1.0,de,...,False,inadmissible,False,5A_39/2007,1,109 II 350;118 II 254,2,None,Tribunale federale Tribunal federal {T 0/2} 5A...,10.5281/zenodo.14867950
4,2C_29/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-27,2007.0,Public,False,2. Public,Public,3.0,de,...,False,rejected,False,2C_29/2007;2C_1/2007,2,131 II 449,1,None,Tribunale federale Tribunal federal {T 0/2} 2C...,10.5281/zenodo.14867950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127472,1C_642/2023,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-30,2024.0,Public,False,1. Public,Public,3.0,de,...,False,rejected,False,1C_642/2023,1,149 II 337;148 II 392;148 V 209;149 II 337;148...,4,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127473,7B_679/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-19,2024.0,Criminal,False,2. Criminal,Criminal,1.0,de,...,True,writeoff,None,7B_679/2024;7B_322/2023;7B_323/2023;7B_324/2023,4,,0,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127474,4A_423/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-17,2024.0,Civil,False,1. Civil,Civil,3.0,fr,...,True,rejected,False,4A_423/2024;4A_458/2022;4A_509/2021;4A_509/202...,9,145 I 239;138 III 46;140 III 86;137 III 580;13...,29,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127475,4A_284/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-17,2024.0,Civil,False,1. Civil,Civil,3.0,fr,...,True,granted,True,4A_284/2024;4A_468/2020;4A_347/2016;4A_468/202...,14,140 III 115;137 I 58;137 II 353;137 III 226;13...,20,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
